In [1]:
# !pip install accelerate  # 모델 학습 속도 향상
# !pip install peft        # LoRA를 포함한 효율적 미세 조정
# !pip install bitsandbytes # 양자화 기술을 통한 메모리 최적화
# !pip install transformers # 트랜스포머 기반 모델 라이브러리
# !pip install datasets     # 데이터셋 로드 및 처리
# !pip install trl          # 트랜스포머 기반 강화 학습
# !pip install pandas       # 데이터 처리 라이브러리

In [2]:
from dotenv import load_dotenv
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from transformers import pipeline

/home/jihun.kim/src/cosera-generative-ai/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

load_dotenv()

True

In [4]:
# !huggingface-cli login --token ${HF_TOKEN}

In [5]:
import os
import sys
import torch

# GPU 사용 가능 여부 확인
device = []
if sys.platform == 'darwin':
    device.append[("mps" if torch.backends.mps.is_available() else "cpu")]
else:
    if torch.cuda.is_available() :
        for ix in range(torch.cuda.device_count()):
            device.append(f"""cuda:{ix}""")
    else:
        device.append("cpu")
       
device

['cuda:0', 'cuda:1']

In [6]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [7]:
ds =  load_dataset("beomi/KoAlpaca-v1.1a", split="train[:5%]")
ds


Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 1058
})

In [8]:
# 데이터 확인
ds[0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

In [9]:
ds = ds.with_format("torch", device=device[0])

In [10]:
def format_example(row):
    return {
        'text': f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
        You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>
 
        {row['instruction']}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>
 
        {row['output']}<|eot_id|>"""
    }
 
# 판다스 데이터프레임을 데이터셋으로 변환하고, 포맷팅 함수 적용
dataset = ds.map(format_example)

In [29]:
base_model = "beomi/Llama-3-KoEn-8B"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map=device[0],
    quantization_config=BitsAndBytesConfig(load_in_8bit=True)
)
 
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
              base_model,
              trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # 시퀀스 패딩에 eos 토큰 사용
tokenizer.padding_side = "right"           # 패딩을 오른쪽에 추가


Loading checkpoint shards: 100%|██████████| 6/6 [00:07<00:00,  1.18s/it]


In [12]:
# LoRA 설정
peft_params = LoraConfig(
    lora_alpha=16,       # LoRA의 스케일링 계수 설정
    lora_dropout=0.1,    # 드롭아웃을 통해 과적합 방지
    r=8,                 # LoRA 어댑터 행렬의 Rank 설정
    bias="none",         # 편향 사용 여부 설정
    task_type="CAUSAL_LM", # 작업 유형 설정 (Causal LM)
    target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj'] # 적용 모듈 설정
)
 
# 모델을 8bit 학습을 위한 상태로 준비. 메모리를 절약하면서도 모델의 성능을 유지할 수 있음
model = prepare_model_for_kbit_training(model, 8)


In [13]:
# PEFT 어댑터 설정을 모델에 적용
model = get_peft_model(model, peft_params)

In [14]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 6815744
all model parameters: 8037076992
percentage of trainable model parameters: 0.08%


In [15]:
# model.to(device[0])

# 학습 파라미터 설정
# training_params = TrainingArguments(
#     output_dir="./results",              # 결과 저장 경로
#     num_train_epochs=10,                 # 학습 에폭 수
#     per_device_train_batch_size=1,       # 배치 사이즈
#     learning_rate=2e-4,                  # 학습률 설정
#     save_steps=1000,                     # 저장 빈도
#     logging_steps=50,                    # 로그 출력 빈도
#     fp16=True                            # 16-bit 부동 소수점 사용 (메모리 절약),
# )

training_params = SFTConfig(
    output_dir="./results",              # 결과 저장 경로
    num_train_epochs=10,                 # 학습 에폭 수
    per_device_train_batch_size=4,       # 배치 사이즈
    learning_rate=2e-4,                  # 학습률 설정
    save_steps=1000,                     # 저장 빈도
    logging_steps=50,                    # 로그 출력 빈도
    fp16=True,                            # 16-bit 부동 소수점 사용 (메모리 절약),
    max_seq_length=512
)


In [16]:
# SFTTrainer를 사용해 학습 실행
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    tokenizer=tokenizer,
    args=training_params
)


/tmp/ipykernel_1469679/1806763960.py:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
Tokenizing train dataset: 100%|██████████| 1058/1058 [00:00<00:00, 1614.20 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/jihun.kim/src/cosera-generative-ai/.venv/lib64/python3.12/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/jihun.kim/src/cosera-generative-ai/.venv/lib64/python3.12/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,1.891800
100,1.695900
150,1.675300
200,1.624900
250,1.623400
300,1.585700
350,1.558200
400,1.565900
450,1.473300
500,1.465500


/home/jihun.kim/src/cosera-generative-ai/.venv/lib64/python3.12/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/jihun.kim/src/cosera-generative-ai/.venv/lib64/python3.12/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=1330, training_loss=1.317376254375716, metrics={'train_runtime': 8878.2355, 'train_samples_per_second': 1.192, 'train_steps_per_second': 0.15, 'total_flos': 2.121489164916818e+17, 'train_loss': 1.317376254375716})

In [30]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)
 
def generate_and_stop(prompt):
    result = pipe(f"{prompt}")[0]['generated_text']
    return result

Device set to use cuda:0


In [32]:
prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>
 
양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>
"""
 
generate_and_stop(prompt)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n \nYou are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n \n양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou are a helpful assistant\nYou ar

In [20]:
trainer.save_model("./results/peft-tuned-Llama-3-KoEn-8B")

In [22]:
peft_tuned_model = "./results/peft-tuned-Llama-3-KoEn-8B"
tuned_model = AutoModelForCausalLM.from_pretrained(
    peft_tuned_model,
    device_map=device[1])

Loading checkpoint shards: 100%|██████████| 6/6 [00:10<00:00,  1.82s/it]


In [25]:
tuned_tokenizer = AutoTokenizer.from_pretrained(
              peft_tuned_model,
              trust_remote_code=True,
              device=device[1])
tuned_tokenizer.pad_token = tokenizer.eos_token  # 시퀀스 패딩에 eos 토큰 사용
tuned_tokenizer.padding_side = "right"           # 패딩을 오른쪽에 추가

In [26]:
tuned_pipe = pipeline(task="text-generation", model=tuned_model, tokenizer=tuned_tokenizer, max_length=256)
 
def generate_and_stop_with_tuned(prompt):
    result = tuned_pipe(f"{prompt}")[0]['generated_text']
    return result

Device set to use cuda:1


In [28]:
generate_and_stop_with_tuned(prompt)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n \nYou are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n \n양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n양파는 식물의 꽃줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 고구마는 줄기에서 뿌리를 내리며, 이 뿌리가 열매를 맺는 것입니다. 반면, 양파는 꽃줄기에서 열리는 열매입니다. 양파는 줄기가 없고, 잎과 꽃줄기가 바로 연결되어 있습니다. 이러한 차이점을 잘 기억해두시면 좋을 것 같습니다.�\n�assistant�\n \n고구마는 뿌리인가요? \n고구마는 줄기에서 뿌리를 내리며, 이 뿌리가 열매를 맺습니다. 줄기에서 뻗어나온 고구마는 땅속에서 자라며, 이때 자라는 것이 뿌리입니다. 따라서 고구마는 식물의 뿌리 부분입니다. 양파는 꽃줄기에서 열리는 열매입니다. 이러한'